In [136]:
import pandas as pd
import numpy as np

In [137]:
tweets = pd.read_csv("twitter_sentiments_10_au_15_.csv")

In [138]:
tweets["count_pos"] = tweets["count"]*tweets["pos"]

In [139]:
tweets = tweets.groupby("candidat").sum()["count_pos"].reset_index()

In [140]:
tweets["bloc"] = tweets["candidat"].map(lambda x : "gauche" if x in ["arthaud", "poutou", "macron", "hamon", "melenchon"] else x)

In [141]:
tweets["bloc"] = tweets["bloc"].map(lambda x : "droite" if x in ["cheminade", "fillon", "nda", "lassalle", "asselineau"] else x)

In [142]:
print(tweets)

      candidat  count_pos    bloc
0      arthaud        1.0  gauche
1   asselineau       73.0  droite
2    cheminade        5.0  droite
3       fillon     7714.0  droite
4        hamon     2487.0  gauche
5     lassalle       27.0  droite
6       le pen     2192.0  le pen
7       macron     5832.0  gauche
8    melenchon     5780.0  gauche
9          nda     1287.0  droite
10      poutou      597.0  gauche


In [143]:
sum_blocs = tweets.groupby("bloc").sum().reset_index()
tweets = tweets.merge(sum_blocs, how="left", on="bloc")

In [144]:
tweets

,candidat,count_pos_x,bloc,count_pos_y
0,arthaud,1.0,gauche,14697.0
1,asselineau,73.0,droite,9106.0
2,cheminade,5.0,droite,9106.0
3,fillon,7714.0,droite,9106.0
4,hamon,2487.0,gauche,14697.0
5,lassalle,27.0,droite,9106.0
6,le pen,2192.0,le pen,2192.0
7,macron,5832.0,gauche,14697.0
8,melenchon,5780.0,gauche,14697.0
9,nda,1287.0,droite,9106.0


In [145]:
tweets["%pos"] = tweets["count_pos_x"]*100/tweets["count_pos_y"]

In [146]:
print(tweets)

      candidat  count_pos_x    bloc  count_pos_y        %pos
0      arthaud          1.0  gauche      14697.0    0.006804
1   asselineau         73.0  droite       9106.0    0.801669
2    cheminade          5.0  droite       9106.0    0.054909
3       fillon       7714.0  droite       9106.0   84.713376
4        hamon       2487.0  gauche      14697.0   16.921821
5     lassalle         27.0  droite       9106.0    0.296508
6       le pen       2192.0  le pen       2192.0  100.000000
7       macron       5832.0  gauche      14697.0   39.681568
8    melenchon       5780.0  gauche      14697.0   39.327754
9          nda       1287.0  droite       9106.0   14.133538
10      poutou        597.0  gauche      14697.0    4.062053


In [147]:
tweets["pred_blocs"] = [40.25, 25.96, 25.96, 25.96, 40.25, 25.96, 23.46, 40.25, 40.25, 25.96, 40.25]
tweets["pred_pond_twitter"] = tweets["%pos"]*tweets["pred_blocs"]/100
tweets["transferts_bayrou_%"] = [0, 0, 0, 20, 7, 2, 4, 50, 12, 4, 1]
tweets["pred_pond_twitter"] = tweets["pred_pond_twitter"]+10.31*tweets["transferts_bayrou_%"]/100

In [148]:
tweets[["candidat", "pred_pond_twitter"]]

,candidat,pred_pond_twitter
0,arthaud,0.002739
1,asselineau,0.208113
2,cheminade,0.014254
3,fillon,24.053592
4,hamon,7.532733
5,lassalle,0.283173
6,le pen,23.872400
7,macron,21.126831
8,melenchon,17.066621
9,nda,4.081467


In [149]:
google = pd.read_excel("google_trends_10_au_15_avril.xlsx")

In [150]:
google = google.mean().reset_index()

In [151]:
google["index"] = google["index"].map(lambda x : " ".join(x.split()[1:]).lower().replace("dupont-aignan", "nda").replace("é", "e"))

In [152]:
tweets = tweets.merge(google, how="left", left_on="candidat", right_on="index")
tweets.drop("index", axis=1, inplace=True)

In [153]:
tweets.rename(columns={0:"recherches_google"}, inplace=True)

In [154]:
google_bloc = tweets.groupby("bloc").sum()["recherches_google"].reset_index()
tweets = tweets.merge(google_bloc, how="left", on="bloc")

In [155]:
tweets["pred_pond_google"] = tweets["pred_blocs"]*tweets["recherches_google_x"]*100/tweets["recherches_google_y"]/100

In [156]:
tweets["pred_pond_google"] = tweets["pred_pond_google"]+10.31*tweets["transferts_bayrou_%"]/100


In [158]:

tweets[["candidat", "pred_pond_google", "pred_pond_twitter"]]


,candidat,pred_pond_google,pred_pond_twitter
0,arthaud,0.871428,0.002739
1,asselineau,3.343785,0.208113
2,cheminade,1.118323,0.014254
3,fillon,18.956130,24.053592
4,hamon,1.901209,7.532733
5,lassalle,3.098928,0.283173
6,le pen,23.872400,23.872400
7,macron,18.258696,21.126831
8,melenchon,23.066913,17.066621
9,nda,2.123434,4.081467


In [ ]:
0.5, 1, 0.33, 19.41, 8, 1.08, 22.42, 23.25, 19.25, 3.5, 1.66